In [29]:
# imports
import os
import sys

# Get the directory of the current file
current_dir = os.path.dirname(os.path.realpath('__file__'))

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
# Add the current directory and parent directory to the system path
sys.path.insert(0, current_dir)
sys.path.insert(0, parent_dir)
import numpy as np              # Python's standard numerical library
import matplotlib.pyplot as plt # Python's scientific visualization library
from pixel_setter import play_video
from scipy.ndimage import uniform_filter
import importlib
from EMA_functions import *
import matplotlib.animation as animation
from DIC_functions import *
import glob
import ast
# import re


# Import test data
df_file_description = pd.read_csv('I:/My Drive/PHD/HSC/file_descriptions_wEMA.csv')
# Back up the data
df_file_description.to_csv('I:/My Drive/PHD/HSC/file_descriptions_wEMA_backup.csv')

# List all files
files = glob.glob('D:/thijsmas/HSC/**/*.cihx', recursive=True)


In [30]:
file = files[30]
name_video = os.path.basename(file)
root_video = os.path.dirname(file)
df_filtered = df_file_description[df_file_description['filename'].isin([name_video])]
indices = df_filtered.index

prey_ij = ast.literal_eval(df_filtered['prey_ij'].item())
spider_ij = ast.literal_eval(df_filtered['spider_ij'].item())
peak_n = ast.literal_eval(df_filtered['peak_n'].item())
peak_F = ast.literal_eval(df_filtered['peak_F'].item())
peak_F_threshold = ast.literal_eval(df_filtered['peak_F_threshold'].item())
shift = ast.literal_eval(df_filtered['shift'].item())
d_lim = df_filtered['d_lim'].item()
test_number = int(df_filtered['test_number'].item())
nut_idx = int(df_filtered['nut_idx'].item())
smooth_lim = df_filtered['smooth_lim'].item()
max_drift = df_filtered['max_drift'].item()
max_end_drift = df_filtered['max_end_drift'].item()
EMA_structure = EMA_Structure(name_video)
EMA_structure.open_impact_data()
video = EMA_structure.open_video(add_extension = False)
EMA_structure.set_params(peak_n = peak_n, peak_F = peak_F, peak_F_threshold = peak_F_threshold, prey_ij = prey_ij, spider_ij = spider_ij, shift = shift, d_lim = d_lim, test_number = test_number, nearest_nut_index = nut_idx, smooth_lim = smooth_lim, max_drift = max_drift, max_end_drift = max_end_drift)
EMA_structure.set_params(FN0 = 41.2, reaction_time = 0.1) # FN0 from Lott: Prey localization in spider orb webs using modal vibration analysis. reaction_time from (Klärner and Barth1982)
DIC_structure = DIC_Structure(file)
video = DIC_structure.video
df = DIC_structure.list_test_data(test_range = range(1, 100), robostness_check = False)
last_row = df.iloc[-1]
EMA_structure.tp, EMA_structure.d = DIC_structure.join_results([test_number])
td = EMA_structure.d +  EMA_structure.tp.reshape(len(EMA_structure.tp),1,2)
EMA_structure.initialize_signals()
EMA_structure.initialize_displacement(idx='all', dir='xy')

D:/thijsmas/HSC\ecc0\web1\Full_web_ecc0_new2_Floc12_v1_S01\Full_web_ecc0_new2_Floc12_v1_S01.cihx
Available filter methods: eig0, eig1, harris, trigs, harmonic_mean, eig_theta, eig_theta_off, eig0_test, eig1_test


c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\pyMRAW.py:104: UserWarning: Clipped footage! (Total frame: 8000, Original total frame: 87371)
  warnings.warn('Clipped footage! (Total frame: {}, Original total frame: {})'.format(cih['Total Frame'], cih['Original Total Frame'] ))


In [31]:
df_filtered.columns

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'filename', 'path', 'ecc',
       'loc', 'mass', 'taut/loose', 'impact/pluck', 'version', 'n_frames',
       'fps', 'Oversatured pixels', 'Undersatured pixels', 'prey_ij',
       'spider_ij', 'peak_n', 'nut_idx', 'd_lim', 'smooth_lim', 'max_drift',
       'max_end_drift', 'lower', 'upper', 'pol_order_high', 'added_damping',
       'near_zero_threshold', 'approximate_height', 'peak_F',
       'peak_F_threshold', 'shift', 'test_number', 'higher', 'spider_ij_d',
       'prey_ij_d'],
      dtype='object')

In [32]:
max_d = np.max(np.abs(EMA_structure.displacements_raw[EMA_structure.nearest_nut_index,:,0]))
first_zero_id_cam = EMA_structure.find_signal_start(EMA_structure.displacements_raw[EMA_structure.nearest_nut_index,:,0], treshold=0.08, approximate_height = max_d*.5, approximate_distance=100000)
first_zero_id_force = EMA_structure.find_signal_start(EMA_structure.force_raw, peak_n=peak_n, treshold=0.05, approximate_height = peak_F_threshold)
# first_zero_id_force = EMA_structure.find_signal_start(EMA_structure.force_raw, peak_n=peak_n, treshold=0.05)
zeros_camera    = EMA_structure.greatest_common_divisor(EMA_structure.fs_camera, EMA_structure.fs_force)
zeros_force     = EMA_structure.n_samples_camera_to_force(zeros_camera)

# Shift and align signals
EMA_structure.t_camera = EMA_structure.shift_time(EMA_structure.t_camera_raw, EMA_structure.t_camera_raw[first_zero_id_cam-zeros_camera])
EMA_structure.t_force = EMA_structure.shift_time(EMA_structure.t_force_raw, EMA_structure.t_force_raw[first_zero_id_force-zeros_force])

# Clip signals
EMA_structure.t_camera          = EMA_structure.clip_signal_before(EMA_structure.t_camera,first_zero_id_cam-zeros_camera)
# EMA_structure.displacements     = EMA_structure.clip_signal_before(EMA_structure.displacements_raw, first_zero_id_cam-zeros_camera)
EMA_structure.displacements_x     = EMA_structure.clip_signal_before(EMA_structure.displacements_raw[:,:,1], first_zero_id_cam-zeros_camera)
EMA_structure.displacements_y     = EMA_structure.clip_signal_before(EMA_structure.displacements_raw[:,:,0], first_zero_id_cam-zeros_camera)
EMA_structure.t_force           = EMA_structure.clip_signal_before(EMA_structure.t_force, first_zero_id_force-zeros_force)
EMA_structure.force             = EMA_structure.clip_signal_before(EMA_structure.force_raw, first_zero_id_force-zeros_force)

last_id_force, last_id_camera   = EMA_structure.find_last_common_time_ids(EMA_structure.t_camera, EMA_structure.t_force)
EMA_structure.t_camera          = EMA_structure.clip_signal_after(EMA_structure.t_camera, last_id_camera)
# EMA_structure.displacements     = EMA_structure.clip_signal_after(EMA_structure.displacements, last_id_camera)
EMA_structure.displacements_x     = EMA_structure.clip_signal_after(EMA_structure.displacements_x, last_id_camera)
EMA_structure.displacements_y     = EMA_structure.clip_signal_after(EMA_structure.displacements_y, last_id_camera)
EMA_structure.t_force           = EMA_structure.clip_signal_after(EMA_structure.t_force, last_id_force)
EMA_structure.force             = EMA_structure.clip_signal_after(EMA_structure.force, last_id_force)

# Zero Force after impact
EMA_structure.force = EMA_structure.zero_signal_treshold(EMA_structure.force, 0.1)

In [33]:
%matplotlib qt
fig, ax = plt.subplots(2, 1, figsize=(10, 10))
ax[0].set_title('old approach')
ax[0].plot(EMA_structure.t_camera, EMA_structure.displacements_y[EMA_structure.nearest_nut_index])
ax[1].plot(EMA_structure.t_force, EMA_structure.force)

In [21]:
from scipy.interpolate import interp1d
Input_signal = EMA_structure.force_raw
Input_fps    = EMA_structure.fs_force
Output_signal=  EMA_structure.displacements_raw[EMA_structure.nearest_nut_index,:,0] / np.max(EMA_structure.displacements_raw[EMA_structure.nearest_nut_index,:,0])
Output_fps   = EMA_structure.fs_camera
dt = 1 / Input_fps

start_id_force = np.argmax(Input_signal) - 1000
start_id_cam = np.argmax(Output_signal) - 1000

Input_signal = Input_signal[first_zero_id_force-1000:]
first_zero_id_cam = EMA_structure.find_signal_start(EMA_structure.displacements_raw[EMA_structure.nearest_nut_index,:,0], treshold=0.08, approximate_height = max_d*.5, approximate_distance=100000)
Output_signal = Output_signal[first_zero_id_cam-10:]

# Create time arrays for input and output signals
t_input = np.arange(len(Input_signal)) / Input_fps
t_output = np.arange(len(Output_signal)) / Output_fps
sig_end = np.argmin(np.abs(t_input - t_output[-1]))

Input_signal = Input_signal[:sig_end]
t_input = t_input[:sig_end]

Input_signal_test = np.zeros_like(t_input)
Input_signal_test[500] = 1

# Interpolate the output signal to match the input signal's time points
interpolator = interp1d(t_output, Output_signal, kind='linear', fill_value="extrapolate")
Output_signal_interpolated = interpolator(t_input)

fig, ax = plt.subplots(3, 1, figsize=(10, 10))
ax[0].plot(t_input, Input_signal)
ax[1].plot(t_input, Output_signal_interpolated)


n = len(Input_signal)
freq = np.fft.rfftfreq(n, dt)


Displacement = np.fft.rfft(Output_signal_interpolated, n=n, axis=0) / n
Force = np.fft.rfft(Input_signal, n=n, axis=0) / n

# As we have effectively discarded the negative frequency spectra, the positive frequency components are doubled
Displacement[1:] *= 2
Force[1:] *= 2

T = t_input[-1] # acquisation period

S_xx = 1/T * np.conj(Displacement) * Displacement
S_ff = 1/T * np.conj(Force) * Force
S_xf = 1/T * np.conj(Displacement) * Force
S_fx = 1/T * np.conj(Force) * Displacement

# S_xx = np.mean(S_xx, axis=0)
# S_ff = np.mean(S_ff, axis=0)
# S_xf = np.mean(S_xf, axis=0)
# S_fx = np.mean(S_fx, axis=0)

H_1 = S_fx / S_ff
H_2 = S_xx / S_xf

coherence = H_1 / H_2

In [22]:
fig, ax = plt.subplots(2, figsize=(20, 5))

ax[0].semilogy(freq, np.abs(H_1))
ax[1].plot(freq, np.angle(H_1))

# ax[1].set_ylim(0, 1.2)
ax[0].set_xlim(0, 30)
ax[1].set_xlim(0, 30)

ax[0].set_ylabel('Accelerance [m/s$^2$/N]')
ax[1].set_ylabel('Phase [rad]')
ax[1].set_xlabel('Frequency [Hz]');

np.angle(H_1)[0], np.pi

(3.141592653589793, 3.141592653589793)

In [ ]:

from scipy.signal import correlate
from scipy.signal import convolve

Input_signal = EMA_structure.force_raw
Input_fps    = EMA_structure.fs_force
Output_signal=  EMA_structure.displacements_raw[EMA_structure.nearest_nut_index,:,0] / np.max(EMA_structure.displacements_raw[EMA_structure.nearest_nut_index,:,0])
Output_fps   = EMA_structure.fs_camera

start_id_force = np.argmax(Input_signal) - 1000
start_id_cam = np.argmax(Output_signal) - 1000

Input_signal = Input_signal[first_zero_id_force-1000:]
first_zero_id_cam = EMA_structure.find_signal_start(EMA_structure.displacements_raw[EMA_structure.nearest_nut_index,:,0], treshold=0.08, approximate_height = max_d*.5, approximate_distance=100000)
Output_signal = Output_signal[first_zero_id_cam-10:]

# Create time arrays for input and output signals
t_input = np.arange(len(Input_signal)) / Input_fps
t_output = np.arange(len(Output_signal)) / Output_fps
sig_end = np.argmin(np.abs(t_input - t_output[-1]))

Input_signal = Input_signal[:sig_end]
t_input = t_input[:sig_end]

Input_signal_test = np.zeros_like(t_input)
Input_signal_test[500] = 1

# Interpolate the output signal to match the input signal's time points
interpolator = interp1d(t_output, Output_signal, kind='linear', fill_value="extrapolate")
Output_signal_interpolated = interpolator(t_input)

# # Calculate the cross-correlation between the input and output signals
cross_correlation = correlate(Input_signal, Output_signal_interpolated, mode='full')[:len(t_input)]

# # Convolve the output signal with the input signal
response = np.convolve(Input_signal_test, Output_signal_interpolated, mode='full')[:len(t_input)]
# convolved_signal = convolve(Input_signal_test, Output_signal_interpolated, mode='full')[:len(t_input)]

fig, ax = plt.subplots(3, 1, figsize=(10, 10))
ax[0].set_title('new approach')
ax[0].plot(t_input, Input_signal_test)
ax[0].plot(t_input, Output_signal_interpolated)
ax[1].plot(t_input, Input_signal_test)
ax[0].set_ylim(-1, 1)
ax[1].plot(t_input, response)
ax[2].plot(t_input, cross_correlation, '.')


In [68]:
t = np.linspace(0, 1, 1000)
irf = np.exp(-5*t) * np.sin(2*np.pi*5*t) # simulated impulse response function

impulse = np.zeros_like(t)
impulse[300] = 1.

In [74]:
irf = np.exp(-5*t_input) * np.sin(2*np.pi*5*t_input)

In [ ]:
%matplotlib inline
plt.figure(figsize=(20, 5))
plt.plot(t_input, irf, label='IRF')
plt.plot(t_input, Input_signal_test, label='impulse excitation')

plt.legend();
plt.xlabel('Time / Total time [/]')
plt.ylabel('Signal amplitude');

In [83]:
response = np.convolve(Input_signal_test, irf, mode='full')[:len(t_input)]

In [ ]:

plt.figure(figsize=(20, 5))
plt.plot(t_input, response, label='system response')
plt.plot(t_input, Input_signal_test, label='impulse excitation')

plt.legend();
plt.xlabel('Time / Total time [/]')
plt.ylabel('Signal amplitude');